CRF trainer using the sklearn_crfsuite package (Python wrapper for CRFSuite): https://sklearn-crfsuite.readthedocs.io/en/latest/

In [16]:
from collections import Counter

import sklearn_crfsuite
from sklearn_crfsuite import metrics

from presidio_evaluator import InputSample
from presidio_evaluator.crf_evaluator import CRFEvaluator
from presidio_evaluator.data_generator import read_synth_dataset

In [ ]:
DATA_DATE = "November 12 2019"

Source a dataset to use for training / testing:

In [ ]:
train_samples = read_synth_dataset("../../data/generated_train_{}.json".format(DATA_DATE))
test_samples = read_synth_dataset("../../data/generated_test_{}.json".format(DATA_DATE))

In [ ]:
train_tagged = [sample for sample in train_samples if len(sample.spans) > 0]
print("Kept {} train samples after removal of non-tagged samples".format(len(train_tagged)))
train_data = InputSample.create_conll_dataset(train_tagged)

test_data = InputSample.create_conll_dataset(test_samples)
test_data.head()

In [ ]:
# Turn every sentence into a list of lists (list of tokens + pos + label)
test_sents=test_data.groupby('sentence')[['text','pos','label']].apply(lambda x: x.values.tolist())
train_sents=train_data.groupby('sentence')[['text','pos','label']].apply(lambda x: x.values.tolist())


Create features for CRF

In [ ]:
CRFEvaluator.sent2features(train_sents[0])[0]

In [ ]:
%%time
X_train = [CRFEvaluator.sent2features(s) for s in train_sents]
y_train = [CRFEvaluator.sent2labels(s) for s in train_sents]

X_test = [CRFEvaluator.sent2features(s) for s in test_sents]
y_test = [CRFEvaluator.sent2labels(s) for s in test_sents]

In [ ]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Save trained model to pickle

In [ ]:
import pickle
with open("../../models/crf.pickle",'wb') as f:
    data = pickle.dump(crf, f,protocol=pickle.HIGHEST_PROTOCOL)
    

Open saved model

In [ ]:
with open("../../models/crf.pickle", 'rb') as f:
    crf = pickle.load(f)

Extract info and predictions from model

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
labels

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

In [ ]:
## predict one:
y_5_pred = crf.predict([X_test[5]])
y_5_pred[0]

In [ ]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

Model explainability

In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])